# XGBoost integration
Here, we present XGBoost integration with Neptune that lets you automatically log metrics (train, eval), save trained model to Neptune and much more.

Usage is easy: just pass `neptune_callback` to training function like any other xgboost callback.

You can log multiple data types:

    * Log metrics (train and eval) after each boosting iteration.
    * Log model (Booster) to Neptune after last boosting iteration.
    * Log feature importance to Neptune as image after last boosting iteration.
    * Log visualized trees to Neptune as images after last boosting iteration.

## Resources
* Open source implementation is on [GitHub](https://github.com/neptune-ai/neptune-contrib/blob/master/neptunecontrib/monitoring/xgboost_monitor.py),
* Use case / tutorial: https://docs.neptune.ai/integrations/xgboost.html,
* Try integration in action right now. Simply run this Notebook top to bottom. It works outside-the-box :)

## Install dependencies
This demo requires few Python libs. Let's install them.

In [ ]:
!pip install neptune-client pandas xgboost sklearn matplotlib graphviz

In [ ]:
import neptune
import pandas as pd
import xgboost as xgb
from sklearn.datasets import load_boston
from sklearn.model_selection import train_test_split

# here you import `neptune_calback` that does the magic (the open source magic :)
from neptunecontrib.monitoring.xgboost_monitor import neptune_callback

## Set project
For this demonstration, I use public user: `neptuner`, who has `ANONYMOUS` token .

Thanks to this you can run this code as is and see results in Neptune :)

In [ ]:
neptune.init('shared/XGBoost-integration',
             api_token='ANONYMOUS')

## Prepare data for XGBoost training

In [ ]:
boston = load_boston()
data = pd.DataFrame(boston.data)
data.columns = boston.feature_names
data['PRICE'] = boston.target
X, y = data.iloc[:,:-1], data.iloc[:,-1]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=102030)

dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test, label=y_test)

## Prepare params

In [ ]:
params = {'max_depth': 5,
          'eta': 0.5,
          'gamma': 0.1,
          'silent': 1,
          'subsample': 1,
          'lambda': 1,
          'alpha': 0.35,
          'objective': 'reg:linear',
          'eval_metric': ['mae', 'rmse']}
watchlist = [(dtest, 'eval'), (dtrain, 'train')]
num_round = 20

## Train model using `xgb.train()`

Example experiment:

In [ ]:
neptune.create_experiment(name='xgb', tags=['train'], params=params)
xgb.train(params, dtrain, num_round, watchlist,
          callbacks=[neptune_callback(log_tree=[0,1,2])])

In [ ]:
neptune.stop()

## Train model using `xgb.cv()`

Example experiment:

In [ ]:
neptune.create_experiment(name='xgb', tags=['cv'], params=params)
xgb.cv(params, dtrain, num_boost_round=num_round, nfold=7,
       callbacks=[neptune_callback(log_tree=[0, 1, 2, 3, 4])])

In [ ]:
neptune.stop()

## Train model using `sklearn` API

Example experiment:

In [ ]:
neptune.create_experiment(name='xgb', tags=['sklearn'], params=params)
reg = xgb.XGBRegressor(**params)
reg.fit(X_train, y_train,
        eval_metric=['mae', 'rmse'],
        eval_set=[(X_test, y_test)],
        callbacks=[neptune_callback(log_tree=[0,1])])

In [ ]:
neptune.stop()